In [ ]:
!pip install ortools

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 2.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.21.12
    Uninstalling protobuf-4.21.12:
      Successfully uninstalled protobuf-4.21.12


In [ ]:
import json
import math
from docplex.mp.model import Model
from time import time

def solve_instance_binary_grid(instance_path: str, solution_path: str):
    
    print("Reading File")

    with open(instance_path, 'r') as f:
        instance = json.load(f)


    Interventions = instance["Interventions"]  # dict of interventions
    T = instance["T"]                         # 1..T (integer)
    Resources = instance.get("Resources", {})
    alpha = float(instance.get("Alpha", 1.0))
    tau = float(instance.get("Quantile", 1.0))  # \tau for the quantile
    Exclusions = instance.get("Exclusions", {})
    Seasons = instance.get("Seasons", {})

    mdl = Model(name="BinaryGridExample")


    print("Creating Variables/Constraints")
    print("STEP")
    start_time = time()
    # ----------------------------------------------------------------
    # 1) Create binary variables x[i,s]: 1 if intervention i starts at time s
    # ----------------------------------------------------------------
    x = {}
    for i, data_i in Interventions.items():
        tmax_i = int(data_i["tmax"])
        x[i] = {}
        for s in range(1, tmax_i + 1):
            x[i][s] = mdl.binary_var(name=f"x_{i}_{s}")

    # Each intervention starts exactly once
    for i, data_i in Interventions.items():
        tmax_i = int(data_i["tmax"])
        mdl.add_constraint(
            mdl.sum(x[i][s] for s in range(1, tmax_i+1)) == 1,
            ctname=f"one_start_{i}"
        )

    print(time()-start_time)
    start_time = time()

    # ----------------------------------------------------------------
# 2) Resource constraints optimization
# ----------------------------------------------------------------
    print("STEP 2")
    for res_name, resource_data in Resources.items():
        # Pre-compute max/min arrays
        max_array = resource_data.get("max", [999999]*T)
        min_array = resource_data.get("min", [-999999]*T)
        
        # Create a dictionary to store resource usage per time period
        resource_usage = {t: [] for t in range(1, T+1)}
        
        # Pre-process resource usage
        for i, data_i in Interventions.items():
            tmax_i = int(data_i["tmax"])
            wdict_res = data_i["workload"].get(res_name, {})
        
            # Group all possible usages by time period
            for s in range(1, tmax_i+1):
                for t_str, s_dict in wdict_res.items():
                    t = int(t_str)
                    usage = s_dict.get(str(s), 0)
                    if usage != 0:  # Only add non-zero usage
                        resource_usage[t].append((i, s, usage))

        # Create constraints only for periods with actual usage
        for t in range(1, T+1):
            if resource_usage[t]:  # Only if there's any usage in this period
                usage_expr = mdl.sum(x[i][s] * usage 
                                for i, s, usage in resource_usage[t])
                
                # Add max constraint if exists
                if "max" in resource_data:
                    mdl.add_constraint(
                        usage_expr <= max_array[t-1],
                        ctname=f"capacity_{res_name}_{t}"
                    )
                
                # Add min constraint if exists
                if "min" in resource_data:
                    mdl.add_constraint(
                        usage_expr >= min_array[t-1],
                        ctname=f"min_capacity_{res_name}_{t}"
                    )

        
    # ----------------------------------------------------------------
    # 3) (Optional) active[i,t] variables for exclusions
    #    active[i,t] = 1 if i is active at time t
    # ----------------------------------------------------------------
    print(time()-start_time)
    start_time = time()

    print("STEP 3")
    active = {}
    for i, data_i in Interventions.items():
        active[i] = {}
        tmax_i = int(data_i["tmax"])
        delta_list = data_i["Delta"]  # durations by start s
        for t in range(1, T+1):
            a_var = mdl.binary_var(name=f"active_{i}_{t}")
            active[i][t] = a_var
            # sum of x[i,s] for those s that "cover" time t
            cover_expr = []
            for s in range(1, tmax_i+1):
                dur_s = delta_list[s-1]
                if s <= t <= s + dur_s - 1:
                    cover_expr.append(x[i][s])
            mdl.add_constraint(
                a_var == mdl.sum(cover_expr),
                ctname=f"def_active_{i}_{t}"
            )

    # Exclusions: example if ex_list = [i1, i2, season_name]
    for ex_name, ex_list in Exclusions.items():
        i1, i2, season_name = ex_list
        if season_name in Seasons:
            for t_str in Seasons[season_name]:
                t_int = int(t_str)
                mdl.add_constraint(
                    active[i1][t_int] + active[i2][t_int] <= 1,
                    ctname=f"excl_{i1}_{i2}_{t_int}"
                )

    # ----------------------------------------------------------------
    # 4) RISK model: build mean_risk[t], quantile Q[t], and "excess"
    #
    #    We assume each i has: data_i["risk"][ str(t) ][ str(s) ] = [scenario0, scenario1, ...]
    #    i.e. an array of scenario risk values if i starts at s, at time t.
    #
    #    We'll parse scenario_count from the data, then for each t,sc build the
    #    total risk[t,sc] as sum over i,s of x[i,s]*( that scenario's risk ).
    # ----------------------------------------------------------------

    print(time()-start_time)
    start_time = time()
    print("STEP 4")
    # Find scenario count more efficiently
    scenario_count = next(
        (len(data_i["risk"][next(iter(data_i["risk"]))][next(iter(data_i["risk"][next(iter(data_i["risk"]))]))])
        for i, data_i in Interventions.items()
        if "risk" in data_i
    ), 1)

    scenario_indices = range(scenario_count)

    # Pre-compute risk values for faster access
    risk_values = {}
    for i, data_i in Interventions.items():
        if "risk" not in data_i:
            continue
        risk_values[i] = {}
        tmax_i = int(data_i["tmax"])
        for t in range(1, T+1):
            t_str = str(t)
            if t_str not in data_i["risk"]:
                continue
            risk_values[i][t] = {}
            for start_s in range(1, tmax_i+1):
                s_str = str(start_s)
                if s_str in data_i["risk"][t_str]:
                    risk_values[i][t][start_s] = data_i["risk"][t_str][s_str]

    # Build risk expressions more efficiently
    risk_t_sc = {}
    mean_risk = {}
    for t in range(1, T+1):
        risk_expressions = []
        for sc in scenario_indices:
            expr = mdl.linear_expr()
            for i in risk_values:
                if t in risk_values[i]:
                    for start_s, scenarios in risk_values[i][t].items():
                        if sc < len(scenarios):
                            expr.add(x[i][start_s] * scenarios[sc])
            risk_t_sc[(t, sc)] = expr
            risk_expressions.append(expr)
        
        # Calculate mean risk directly
        mean_risk[t] = (1.0 / scenario_count) * mdl.sum(risk_expressions)


    print(time()-start_time)
    start_time = time()
    print("STEP 5")

    # ----------------------------------------------------------------
    # 5) Enforce that Q[t] is the tau-quantile using a typical big-M approach:
    #    Introduce Q[t], z[t,sc] in {0,1}, with:
    #      risk_t_sc[t,sc] <= Q[t] + M*z[t,sc]
    #    sum_{sc} z[t,sc] <= floor((1 - tau)*|S|)
    #    => ensures Q[t] >= the (tau)-quantile of {risk_t_sc[t,sc]}.
    # ----------------------------------------------------------------
    Q = {}
    z = {}
    M = 1e7  # Big enough upper bound on risk
    # If your data can have bigger risk, enlarge M accordingly.

    # For each time t, we define Q[t], plus binary z[t,sc]
    for t in range(1, T+1):
        Q[t] = mdl.continuous_var(name=f"Q_{t}")
        for sc in scenario_indices:
            z[(t,sc)] = mdl.binary_var(name=f"z_{t}_{sc}")
        # sum of z[t,sc] <= (1 - tau)*|S|
        mdl.add_constraint(
            mdl.sum(z[(t,sc)] for sc in scenario_indices) <= math.floor((1.0 - tau)*len(scenario_indices)),
            ctname=f"quantile_card_{t}"
        )
    # risk_t_sc[t,sc] <= Q[t] + M*z[t,sc]
    for t in range(1, T+1):
        for sc in scenario_indices:
            mdl.add_constraint(
                risk_t_sc[(t, sc)] <= Q[t] + M * z[(t,sc)],
                ctname=f"risk_le_quant_{t}_{sc}"
            )
            
    print(time()-start_time)
    start_time = time()
    print("STEP 6")
    

    # ----------------------------------------------------------------
    # 6) Excess[t] = max(0, Q[t] - mean_risk[t]).
    #    We'll do it via a continuous var e[t] >= Q[t] - mean_risk[t] & e[t] >= 0
    # ----------------------------------------------------------------
    e = {}
    for t in range(1, T+1):
        e[t] = mdl.continuous_var(lb=0, name=f"excess_{t}")
        # e[t] >= Q[t] - mean_risk[t]
        mdl.add_constraint(
            e[t] >= Q[t] - mean_risk[t],
            ctname=f"def_excess_{t}"
        )

    print(time()-start_time)
    start_time = time()
    print("STEP 7")
    # ----------------------------------------------------------------
    # 7) Build final objective:
    #    Obj_1 = sum_{t} mean_risk[t]
    #    Obj_2 = sum_{t} e[t]
    #    Obj   = alpha * Obj_1 + (1 - alpha)* Obj_2
    # ----------------------------------------------------------------
    obj1_expr = mdl.sum(mean_risk[t] for t in range(1, T+1))
    obj2_expr = mdl.sum(e[t] for t in range(1, T+1))
    mdl.minimize(alpha * obj1_expr + (1 - alpha) * obj2_expr)

    # ----------------------------------------------------------------
    # Solve the model
    # ----------------------------------------------------------------
    sol = mdl.solve(log_output=True)
    if not sol:
        print("No solution found.")
        return

    # ----------------------------------------------------------------
    # 8) Write out a solution: pick s where x[i][s] == 1
    # ----------------------------------------------------------------
    with open(solution_path, "w") as fsol:
        for i, data_i in Interventions.items():
            tmax_i = int(data_i["tmax"])
            start_chosen = None
            for s in range(1, tmax_i+1):
                if sol.get_value(x[i][s]) > 0.5:
                    start_chosen = s
                    break
            fsol.write(f"{i} {start_chosen}\n")

    print(f"Solution written to {solution_path}.")
    print("Objective value: ", sol.objective_value)
    print("Number of constraints: ", mdl.number_of_constraints)
    print("Number of variables: ", mdl.number_of_variables)
    
    # If you want to see the LP file:
    # print(mdl.export_to_string())

def main():
    instance_file = "/home/kingmrock/Roadef_projet/challenge-roadef-2020-master/C_01.json"
    solution_file = "example_solution.txt"
    solve_instance_binary_grid(instance_file, solution_file)

if __name__ == "__main__":
    main()


Reading File
Creating Variables/Constraints
STEP
0.15203094482421875
STEP 2
STEP 2
1.1774592399597168
STEP 3
0.8755505084991455
STEP 4
50.04890847206116
STEP 5
4.459247827529907
STEP 6
0.060394287109375
STEP 7
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 2 times.
MIP Presolve eliminated 1585 rows and 472 columns.
MIP Presolve modified 39265 coefficients.
Aggregator did 1157 substitutions.
Reduced MIP has 16266 rows, 20123 columns, and 3513361 nonzeros.
Reduced MIP has 20017 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 15.37 sec. (8103.55 ticks)
Probing time = 0.08 sec. (46.81 ticks)
Tried aggregator 2 times.
Detecting symmetries...
MIP Presolve eliminated 9 rows and 0 columns.
MIP Presolve modified 1057 coefficients.
Aggregator did 103 substitutions.
Reduced MIP has 16154 rows, 20020 columns, and 3513280 nonzeros.
Reduced MIP has 19914 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve

In [12]:
#print the number of variables and constraints in the model
print(mdl.number_of_variables, mdl.number_of_constraints)


NameError: name 'mdl' is not defined

# Explanation of the Code

## Imports and Setup
- `import json` and `import math`: used for reading JSON data and mathematical functions.
- `from docplex.mp.model import Model`: imports the CPLEX modeling library for building and solving optimization models. 
- `solve_instance_binary_grid(instance_path: str, solution_path: str)`: defines a function that reads problem data from a JSON file, builds a Mixed Integer Programming model, solves it, and writes a solution to a text file.

## Reading the Input
- The code opens `instance_path` as a JSON file and stores its contents in `instance`.
- Key entries from `instance`:
    - `Interventions`: a dictionary of all interventions
    - `T`: the planning horizon
    - `Resources`: a dictionary of resource data
    - `Alpha` ($\alpha$) and `Quantile` ($\tau$): parameters for the risk-based objective
    - `Exclusions` and `Seasons`: specify pairs of interventions or periods that cannot overlap
- `mdl` is an instance of `Model`, representing our optimization problem.

## Binary Variables ($x_{i,s}$)
- For each intervention $i$ and each possible start time $s \in \{1,\dots,t_{\max i}\}$, the code creates a binary variable $x_{i,s}$.
- A constraint ensures that each intervention $i$ starts *exactly once*, i.e.:
$\sum_{s=1}^{t_{\max i}} x_{i,s} = 1$

## Resource Constraints
- For each resource `res_name` and each time $t$, a capacity constraint ensures the total usage does not exceed the available maximum (`max_array[t]`).
- The usage is summed across all interventions:
$\sum_{i, s} x_{i,s} \times \text{(usage if intervention $i$ starts at $s$ at time $t$)}\Big \leq \text{max capacity}$

## Activity Variables ($\text{active}_{i,t}$)
- $\text{active}_{i,t}$ indicates whether intervention $i$ is active at time $t$
- A logic constraint defines $\text{active}_{i,t}$ as the sum of all $x_{i,s}$ such that $t \in [s,\, s+\delta_{s}-1]$

## Exclusions 
- If two interventions cannot both be active in a certain season, constraints enforce that at most one is active (sum of their $\text{active}$ variables $\leq 1$)

## Risk Expressions
- For each time $t$ and scenario $\text{sc}$, $\text{risk\_t\_sc}(t,\text{sc})$ captures total risk across all interventions started
- $\text{mean\_risk}[t]$ is the average risk at time $t$ across all scenarios

## Quantile Constraints
- $\text{Q}[t]$ is the $\tau$-quantile of the risk distribution at time $t$
- Binary variables $z[t,\text{sc}]$ and a big-$M$ constraint ensure $\text{Q}[t]$ is large enough so at most $(1-\tau)\cdot |S|$ scenarios exceed it

## Excess Variables
- Each $e[t]$ measures how much $\text{Q}[t]$ exceeds the mean risk:
$e[t] \geq \text{Q}[t] - \text{mean\_risk}[t], \quad e[t] \geq 0$

## Objective Function
Two components:
$\text{Obj}_1 = \sum_{t=1}^{T} \text{mean\_risk}[t]$
$\text{Obj}_2 = \sum_{t=1}^{T} e[t]$

Final objective (minimized):
$\alpha \times \text{Obj}_1 + (1 - \alpha) \times \text{Obj}_2$

## Solution and Output
- Model solved with `mdl.solve(log_output=True)`
- For each intervention $i$, finds start time $s$ where $x_{i,s}=1$ and writes $(i,s)$ to `solution_file`


In [3]:
#print the fields of the instance
print(instance.keys())

dict_keys(['Resources', 'Seasons', 'Interventions', 'Exclusions', 'T', 'Scenarios_number', 'Quantile', 'Alpha', 'ComputationTime'])


In [7]:
!pip install gurobipy

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 9.4 MB/s eta 0:00:00ta 0:00:01


In [9]:
#print the number of scenarios
print(instance[SCENARIO_NUMBER])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
